<a href="https://colab.research.google.com/github/THARAKSAIRAM/SIH-2020/blob/master/Language%20Classsification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pip install python_speech_features


  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=4c72d2daf1758365f03af91bef242d7cdf10685d5317ba7bb88b55b63c2f3ded
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
from python_speech_features import mfcc,delta,logfbank 

In [0]:
import numpy
import scipy.io.wavfile
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import wave
import contextlib
from scipy.fftpack import dct
import scipy.io.wavfile as wav

In [5]:
from keras import applications
from keras.layers import GlobalAveragePooling2D,Dropout,Dense,BatchNormalization,Activation,AveragePooling2D,Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.models import load_model

Using TensorFlow backend.


In [0]:
pre_emphasis = 0.97
frame_size = 0.025
frame_stride = 0.01
NFFT = 512
nfilt = 40
num_ceps = 12
cep_lifter =22

In [0]:
def length_audio(folder_name,file_name):
  file_path='/content/'+str(folder_name)+"/"+str(file_name)
  sample_rate, signal = scipy.io.wavfile.read(file_path)
  with contextlib.closing(wave.open(file_path,'r')) as f:
      frames = f.getnframes()
      rate = f.getframerate()
      duration = frames / float(rate)
  return int(duration)

In [0]:
def create_mfcc(folder_name,file_name,start_point):
    file_path ='/content/'+str(folder_name)+"/"+str(file_name)
    sample_rate, signal = scipy.io.wavfile.read(file_path)
    signal = signal[start_point:int(start_point+2 * sample_rate)]   #framing to 2 seconds
    emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1]) #pre-emphasis
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
    pad_signal_length = num_frames * frame_step + frame_length
    z = numpy.zeros((pad_signal_length - signal_length))
    pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
    indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(numpy.int32, copy=False)]
    #After slicing the signal into frames, we apply a window function such as the Hamming window
    frames *= numpy.hamming(frame_length)  #hamming window
    mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
    low_freq_mel = 0
    high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
    mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
    bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])

    filter_banks = numpy.dot(pow_frames, fbank.T)
    filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * numpy.log10(filter_banks)  # dB
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13
    (nframes, ncoeff) = mfcc.shape
    n = numpy.arange(ncoeff)
    lift = 1 + (cep_lifter / 2) * numpy.sin(numpy.pi * n / cep_lifter)
    mfcc *= lift  #*
    
    filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)
    mfcc -= (numpy.mean(mfcc, axis=0) + 1e-8)
    
    return filter_banks

In [0]:
def create_model():
  base_model=applications.resnet50.ResNet50(weights=None,include_top=False,input_shape = (40,40,1))
  x = base_model.output
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size=2)(x)
  y = Flatten()(x)
  outputs = Dense(11,activation='softmax',kernel_initializer='he_normal')(y)
  model=Model(inputs=base_model.input,outputs=outputs)
  adam=Adam(lr=0.0001)
  model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [16]:
model = create_model()
model.load_weights('drive/My Drive/language_classificationII.h5')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


ValueError: ignored

In [0]:
def prediction(targ):
    accents = ['bengali','dogri','gujarathi','hindi','malayalam','manipuri','marathi','odisha','tamil','telugu','urdu']
    np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    #np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    l=[]
    out_len = len(targ)
    in_len = len(targ[0])
    for i in range(in_len):
        sumi = 0
        for j in range(out_len):
            sumi+=targ[j][i]
        l.append(sumi)
    print(l)
    maximum = -1
    max_ind = -1
    for i in range(len(l)):
        if(l[i] > maximum):
            maximum = l[i]
            max_ind = i+1
    # print(maximum, max_ind)
    print("Accent: ",accents[max_ind-1])
    for i in l:
        if i!=0:
            per = (i/float(out_len))*100
            if(per>1):
                print("{1:.2f}% -> {0}".format(accents[l.index(i)],per))

In [0]:

def Pre_final_data(folder,files):
    index=0
    
    mfcc=[]
    target=[]
    mfcc_len=length_audio(folder,files)
    data_dic = np.zeros((1+mfcc_len//2,398,40))
    for i in range(0,mfcc_len,2):
        
        mfcc = create_mfcc(folder,files,i)
        for j in range(len(mfcc)):
              for k in range(len(mfcc[j])):
                data_dic[index][j][k] = mfcc[j][k]
        index += 1
        #target.append(mfcc)
    #target=np.asarray()
    
    data_dic_reshaped = data_dic.reshape(data_dic.shape[0],398,40,1)
    targ=model.predict(data_dic_reshaped)    
    print(targ)
    prediction(targ)

In [0]:
folder='drive/My Drive/testing'
file='test_krish.wav'
print(Pre_final_data(folder,file))

[[0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]]
[2.2084815313322403e-11, 7.999998688697815, 4.4749546290745457e-08, 3.181414143628783e-12, 4.123314560899871e-07, 1.3797249069158815e-12, 5.236921528583248e-14, 7.840901155020674e-07, 4.7221957107507196e-15, 7.553454715002184e-12, 8.675708823344815e-16]
Accent:  dogri
100.00% -> dogri
None


23650
23650


In [0]:
from keras.models import load_model
model.save('language_classification.h5')

In [0]:
x_test = X_test.reshape(X_test.shape[0],398,40,1)
y_pred=model.predict(x_test)

In [0]:
print(y_pred[0])

[2.1934635e-05 9.9977022e-01 1.1724382e-07 3.0121672e-08 8.0185441e-07
 2.6898798e-07 4.6595054e-05 2.0830630e-05 3.5584791e-05 1.6739827e-07
 1.0348346e-04]


In [0]:
import keras
# from sklearn.metrics import accuracy_score
# print(accuracy_score(Y_test, y_pred))
acc=0
for i in range(len(Y_test)):
  count=0
  for j in range(len(Y_test[i])):
    if(Y_test[i][j]==round(y_pred[i][j],0)):
      count+=1
  if(count==11):
    acc+=1
print("Accuracy is",(acc/len(Y_test))*100)

Accuracy is 99.35734821579571


In [0]:
###########Testing manual#############

In [0]:
from keras.layers import GlobalAveragePooling2D,Dropout,Dense,BatchNormalization,Activation,AveragePooling2D,Flatten
from keras.models import Model
from keras import applications
from keras import applications
from keras.models import Sequential,Model
from keras.layers import Convolution2D, MaxPooling2D


In [0]:
def create_model():
  base_model=applications.resnet50.ResNet50(weights=None,include_top=False,input_shape = (40,40,1))
  x = base_model.output
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = AveragePooling2D(pool_size=2)(x)
  y = Flatten()(x)
  outputs = Dense(11,activation='softmax',kernel_initializer='he_normal')(y)
  rnet=Model(inputs=base_model.input,outputs=outputs)
  model = Sequential()
  model.add(Convolution2D(1,(3,3),padding='same',activation = 'relu',input_shape = (398,40,1)))
  model.add(MaxPooling2D(pool_size=(2, 1)))
  model.add(MaxPooling2D(pool_size=(2, 1)))
  model.add(MaxPooling2D(pool_size=(2, 1)))
  model.add(Convolution2D(1, 2, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
  model.add(rnet)
  adam=Adam(lr=0.0001)
  model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
  return model


In [0]:
def prediction(targ):
    accents = ['bengali','dogri','gujarathi','hindi','malayalam','manipuri','marathi','odisha','tamil','telugu','urdu']
    np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    #np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})
    l=[]
    out_len = len(targ)
    in_len = len(targ[0])
    for i in range(in_len):
        sumi = 0
        for j in range(out_len):
            sumi+=targ[j][i]
        l.append(sumi)
    print(l)
    maximum = -1
    max_ind = -1
    for i in range(len(l)):
        if(l[i] > maximum):
            maximum = l[i]
            max_ind = i+1
    # print(maximum, max_ind)
    print("Accent: ",accents[max_ind-1])
    for i in l:
        if i!=0:
            per = (i/float(out_len))*100
            if(per>1):
                print("{1:.2f}% -> {0}".format(accents[l.index(i)],per))
                print(accents[l.index(i)])

In [0]:

def Pre_final_data(folder,files):
    index=0
    data_dic = np.zeros((23650,398,40))
    mfcc=[]
    target=[]
    mfcc_len=length_audio(folder,files)
    
    for i in range(0,mfcc_len,2):
        
        mfcc = create_mfcc(folder,files,i)
        for j in range(len(mfcc)):
              for k in range(len(mfcc[j])):
                data_dic[index][j][k] = mfcc[j][k]
        index += 1
        #target.append(mfcc)
    #target=np.asarray()
    
    data_dic_reshaped = data_dic.reshape(data_dic.shape[0],398,40,1)
    md = create_model()
    md.load_weights('drive/My Drive/language_classificationII.h5')
    targ=md.predict(data_dic_reshaped)    
    print(targ)
    prediction(targ)

In [41]:
folder='drive/My Drive/testing'
file='test_krish.wav'
print(Pre_final_data(folder,file))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (2, 1), activation="relu", use_bias=False)`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 1), activation="relu", use_bias=False)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 1), activation="relu", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 1), activati

[[0.0 0.0 0.0 ... 0.0 0.0 1.0]
 [0.0 0.0 0.0 ... 0.0 0.0 1.0]
 [0.0 0.0 0.0 ... 0.0 0.0 1.0]
 ...
 [0.2 0.8 0.0 ... 0.0 0.0 0.0]
 [0.2 0.8 0.0 ... 0.0 0.0 0.0]
 [0.2 0.8 0.0 ... 0.0 0.0 0.0]]
[3994.8042896813554, 19253.42237205195, 8.167068315153939, 42.37557720416251, 0.4861113382798976, 6.550523406867796, 0.18227765817664476, 0.31076717014263977, 147.4658810468468, 1.331657541471941, 194.90236069448292]
Accent:  dogri
16.89% -> bengali
bengali
81.41% -> dogri
dogri
None


SyntaxError: ignored